# Activity #1: yt

In [ ]:
# first install yt!
#!pip install yt

# now import!
import yt

We'll use a dataset originally from the yt hub: http://yt-project.org/data/

Specifically, we'll use the IsolatedGalaxy dataset: http://yt-project.org/data/IsolatedGalaxy.tar.gz

In [ ]:
# now, lets grab a dataset & upload it

# here's where mine is stored (in data)
ds = yt.load("/Users/jillnaiman1/data/IsolatedGalaxy/galaxy0030/galaxy0030")

This will be a bit of a repeat of a few weeks ago, but here we go!

In [ ]:
# print out various stats of this dataset
ds.print_stats()
# this is basically telling us something about the 
#  number of data points in the dataset
#  don't worry if you don't know what levels, grids or cells are at this point
#  we'll get to it later

In [ ]:
# same thing with field list, its cool if some of these look less familiar then others
ds.field_list
ds.derived_field_list

In [ ]:
# this is a 3D simululation of a galaxy, lets check out some stats about the box
ds.domain_right_edge, ds.domain_left_edge
# what this is saying is the box goes from (0,0,0) to (1,1,1) in "code_length" units
#  basically, this is just a normalized box

In [ ]:
# you can also do fun things like print out max & min densities
ds.r[:].max("density"), ds.r[:].min("density")
# the above is for the whole box

In [ ]:
# we can also ask where the maximum density is in this simulation box
ds.r[:].argmax("density")
# so this gives us x/y/z positions for where the maximum
#  density is

In [ ]:
# ok, lets make a quick plot 1/2 down the z-direction

# if the plot is too big for class try:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [3, 3]


p = ds.r[:, :, 0.5].plot("density")

In [ ]:
# lets zoom
p.zoom(10)

In [ ]:
# so, unless you're an astronomer you might be a little confused about these "kpc" units
# but yt allows us to change them!  Behold cool yt units things:
(yt.units.kpc).in_units("cm")
# so we have now changed these weird kpc units 

In [ ]:
# yt also can do cool things with units like, yt units
#  figures out some math stuff like, making things 
#  into cubed cm
(yt.units.kpc**3).in_units("cm**3")

In [ ]:
# so lets set some units of our plot!
#  lets change the units of density from g/cm^3 to kg/m^3
p.set_unit("density","kg/m**3")

In [ ]:
# we can also include annotations on this plot
p.annotate_velocity()
# this shows how material is moving in this simulation
#  this is shown with velocity vectors

In [ ]:
# we can combine some of our coding around finding
#  max values of density and combine with some 
# region plots

# lets project the maximum density along the z axis
#  i.e. lets make a plot of the maximum density 
#  along the z-axis of our plot
p2 = ds.r[:].max("density", axis="z").plot()

In [ ]:
# we can zoom this as well
p2.zoom(10)
# if we scroll back up we can see that there is 
#  indeed a different between this and our slice plot
#  here, we are much more "smeared" since we're picking 
# only the max density -> everything looks brighter

In [ ]:
# we can also do plots based on region selection 
#  but over specific values of z (and x & y)
# if we recall our box goes from 0->1 in each
#  x/y/z direction, we can plot a zoom in 
# like so:
p = ds.r[0.1:0.9, 0.1:0.9, 0.55:0.65].max("density", axis="z").plot()
# so, this shows the maximum density but only in a thin slice 
#  of the z-axis which is offset from the center

# sicne the galaxy lives at the center, and is the highest
#  density gas region, it makes sense that our densities
# are lower and our features look different

# more "fuzzy ball" outside of the galaxy then 
#  gas flowing onto a galaxy disk

In [ ]:
# lets redo the same plot but for the temperature of the gas:
p = ds.r[0.1:0.9, 0.1:0.9, 0.55:0.65].mean("temperature", axis="z").plot()

In [ ]:
# we might want to highlight the temperature of the most dense regions
#  why?  well maybe we want to, instead of depicting the staight 
# temperature, we want to depict the temperature 
# of the *majority of the gas*
#  we can do this by specifying a "weight" in our projection:
p = ds.r[0.1:0.9, 0.1:0.9, 0.55:0.65].mean("temperature", weight="density", axis="z").plot()

In [ ]:
# so why is there this blocky structure?  In space, we don't see cubes around galaxies... 
#  yet anyway...

# this is becuase this is a simulation of a galaxy, not an actual galaxy
#  we can show why this might be by plotting the "grids" of this simulation
# over this thing:
p.annotate_grids()
# from this we can see that our grids sort of align where
#  the temperature looks funny

# this is a good indicator that we have some numerical
#  artifacts in our simulation

In [ ]:
# ok!  lets try some more analysis-like plots
# some of the helpful yt included plots is
ds.r[:].profile("density", "temperature").plot()
# so this is plotting the temperature of the gas 
#  in our simulation, in each binned density

# In our actual simulation, we have temperaturates
# at a variety of densities, and this is 
# usualy the case, so by default what is plotted
#  is the temperature (our 2nd param) plotted
# at each density bin, but weighted by the 
# mass of material (gas) in each cell

In [ ]:
# we can weight by other things, like in this case
# density:
ds.r[:].profile("density", "temperature", weight_field="density").plot()
# so similar shape (since mass and density are related)
#  but a little different

In [ ]:
# we can move this to a 2D plot
# to show the cell mass (as a color)
# as a function of both density and temprature
ds.r[:].profile(["density", "temperature"], "cell_mass", weight_field=None).plot()
# note: we can also do a 3D profile object,
#  but there is currently no associated plot function with it

# Activity #2: Brain data with yt

In [ ]:
# we can also use yt to play with other sorts of data:
import h5py # might have to pip install

In [ ]:
# lets read our datafile into something called "scan_data"
with h5py.File("/Users/jillnaiman1/Downloads/single_dicom.h5", "r") as f:
    scan_data = f["/scan"][:]

In [ ]:
# if we recall, we had a weird shape of this data:
scan_data.shape

In [ ]:
# so to import this data into yt to have 
#  yt make images for us, we need to do some formatting with numpy
import numpy as np

In [ ]:
dsd = yt.load_uniform_grid({'scan': scan_data},
                     [36, 512, 512],
                     length_unit = yt.units.cm,
                     bbox = np.array([[0., 10], [0, 10], [0, 10]]),
)

In [ ]:
dsd.r[:].mean("scan", axis="y").plot(); # this takes the mean along the specified axis "y" and plots
# can also do .max or .min

In [ ]:
# note here that the number of fields 
# availabel is much less:
dsd.field_list

In [ ]:
# we can also look at different potions
# of the z-y axis by specifying
# the x-axis
p = dsd.r[0.75,:,:].plot('scan')

# Activity #3: Output images and objects (3D) with yt
Note: we'll do more with 3D objects next week/the last week, but this is a good first view of some cool ways we can output objects with yt

In [ ]:
# lets go back to to our galaxy object
#  and make a surface

# first, we'll cut down to a sphere and check
# that out
sphere = ds.sphere("max", (500.0, "kpc"))
sphere.mean("density", axis="y").plot(); # this takes the mean along the specified axis "y" and plots

In [ ]:
# lets generate a surface of constant density
#  i.e. we'll connect points on a surface
#  where the density has a single value
surface = ds.surface(sphere, "density", 1e-27)

In [ ]:
surface.export_obj('/Users/jillnaiman1/Downloads/myGalFiles',color_field='temperature')
# the above might take a while

In [ ]:
# for checking out our surfaces right here
#http://www2.compute.dtu.dk/projects/GEL/PyGEL/
#!pip install PyGEL3D

In [ ]:
from PyGEL3D import gel
from PyGEL3D import js

# for navigating

js.set_export_mode()
m = gel.obj_load("/Users/jillnaiman1/Downloads/myGalFiles.obj")
viewer = gel.GLManifoldViewer()
viewer.display(m)



# press ESC to quit?  Yes, but then it takes a while so

In [ ]:
# to get rid of the window
del viewer

Now, lets try with an inline viewer

In [ ]:
# Can also display in the notebook
import numpy as np

#js.display(m,wireframe=False)
# comment out after you've run since we'll re-run below

Now lets try with an inline viewer & data colors

In [ ]:
surf_temp = surface['temperature']
surf_temp.shape
# we see that this is infact a long list of values
#  temperatures on each surface *face*

In [ ]:
# if we look at the shape of the object:
m.positions().shape, surf_temp.shape[0]*3
# we see we have (surf_temp.shape)X3 times
#  the number of points in x/y/z
# this is because these are *vertex* values
# so, if we want to color by something, we should use
#  3X the number of faces

In [ ]:
js.display(m, data=np.repeat(np.log10(surf_temp),3),wireframe=False)


We can also process for 3D printing

In [ ]:
surface.export_obj('/Users/jillnaiman1/Downloads/myGalFiles_print',dist_fac=0.001)


## Outputing images for things like clothing

In [ ]:
p = ds.r[:, :, 0.5].plot("density")

In [ ]:
p.zoom(20)

In [ ]:
myImage = p.frb # fixed resoltuion binary

In [ ]:
# we can then grab a simple image array
plt.imshow(np.array(myImage['density']))

In [ ]:
# or we can turn off labels and grab a lovely image:
p = ds.r[:, :, 0.5].plot("density")
p.zoom(10)

In [ ]:
p.hide_colorbar(); p.hide_axes();

In [ ]:
p

In [ ]:
# save the image
p.save('/Users/jillnaiman1/Downloads/myImage.png')

Now you have a lovely image that you can upload and put on things like sweaters or whatnot.